In [ ]:
#Matthew Trinh

In [2]:
import numpy as np
import pandas as pd
import sqlite3
data = pd.read_csv('http://bit.ly/2cLzoxH')

db = sqlite3.connect(':memory:')
data.to_sql('data', db)
cursor = db.cursor()
db.row_factory = sqlite3.Row
fancyCursor = db.cursor()

data

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,1987,9216418.0,Africa,62.351,706.157306
1700,Zimbabwe,1992,10704340.0,Africa,60.377,693.420786
1701,Zimbabwe,1997,11404948.0,Africa,46.809,792.449960
1702,Zimbabwe,2002,11926563.0,Africa,39.989,672.038623


In [2]:
cursor.execute('''
    SELECT *
    FROM data
    LIMIT 5
''')

cursor.fetchall()

[(0, 'Afghanistan', 1952, 8425333.0, 'Asia', 28.801, 779.4453145),
 (1, 'Afghanistan', 1957, 9240934.0, 'Asia', 30.331999999999997, 820.8530296),
 (2, 'Afghanistan', 1962, 10267083.0, 'Asia', 31.997, 853.1007099999999),
 (3, 'Afghanistan', 1967, 11537966.0, 'Asia', 34.02, 836.1971382),
 (4, 'Afghanistan', 1972, 13079460.0, 'Asia', 36.088, 739.9811057999999)]

In [3]:
fancyCursor.execute('''
    SELECT *
    FROM data
    LIMIT 5
''')

[dict(row) for row in fancyCursor.fetchall()]

[{'index': 0,
  'country': 'Afghanistan',
  'year': 1952,
  'pop': 8425333.0,
  'continent': 'Asia',
  'lifeExp': 28.801,
  'gdpPercap': 779.4453145},
 {'index': 1,
  'country': 'Afghanistan',
  'year': 1957,
  'pop': 9240934.0,
  'continent': 'Asia',
  'lifeExp': 30.331999999999997,
  'gdpPercap': 820.8530296},
 {'index': 2,
  'country': 'Afghanistan',
  'year': 1962,
  'pop': 10267083.0,
  'continent': 'Asia',
  'lifeExp': 31.997,
  'gdpPercap': 853.1007099999999},
 {'index': 3,
  'country': 'Afghanistan',
  'year': 1967,
  'pop': 11537966.0,
  'continent': 'Asia',
  'lifeExp': 34.02,
  'gdpPercap': 836.1971382},
 {'index': 4,
  'country': 'Afghanistan',
  'year': 1972,
  'pop': 13079460.0,
  'continent': 'Asia',
  'lifeExp': 36.088,
  'gdpPercap': 739.9811057999999}]

## The Biggest Countries

In [4]:
# Filters for year 2007, and only pull the 3 columns of interest.
populations = data[data["year"] == 2007][["country", "continent", "pop"]]

# Sorts by population descending, then groups by continent and gets the first row from each grouping.
groupings = populations.sort_values('pop', ascending = False).groupby('continent').head(1)
print(groupings)

records = [(row["continent"], row["country"], row["pop"]) for (id, row) in groupings.iterrows()]

for (continent, country, population) in records:
    print(f"In {continent}, {country} has a population of {population:,.0f}.")

            country continent           pop
299           China      Asia  1.318683e+09
1619  United States  Americas  3.011399e+08
1139        Nigeria    Africa  1.350312e+08
575         Germany    Europe  8.240100e+07
71        Australia   Oceania  2.043418e+07
In Asia, China has a population of 1,318,683,096.
In Americas, United States has a population of 301,139,947.
In Africa, Nigeria has a population of 135,031,164.
In Europe, Germany has a population of 82,400,996.
In Oceania, Australia has a population of 20,434,176.


In [5]:
fancyCursor.execute('''
    SELECT d.continent, d.country, d.pop
    FROM data d
    INNER JOIN (
      SELECT continent, MAX(pop) AS maxPop
      FROM data
      WHERE year = 2007
      GROUP BY continent
    ) m
      ON d.continent = m.continent AND d.pop = m.maxPop
    ORDER BY pop DESC
''')

records = [(row["continent"], row["country"], row["pop"]) for row in fancyCursor.fetchall()]


for (continent, country, population) in records:
    print(f"In {continent}, {country} has a population of {population:,.0f}.")

In Asia, China has a population of 1,318,683,096.
In Americas, United States has a population of 301,139,947.
In Africa, Nigeria has a population of 135,031,164.
In Europe, Germany has a population of 82,400,996.
In Oceania, Australia has a population of 20,434,176.


## Living conditions

Which countries have had a decrease in their life expectancy when comparing 1952 against 2007?

In [227]:
records1952 = data[data["year"] == 1952][["country","lifeExp"]].reset_index()
records2007 = data[data["year"] == 2007][["country", "lifeExp"]].reset_index()
filt = records1952["lifeExp"] - records2007["lifeExp"] > 0
records = records1952[filt]["country"].to_string(index=False).strip().split()

print(f"The countries whose life expectancy has decreased since 1952 are: {records}.")

Swaziland
The countries whose life expectancy has decreased since 1952 are: ['Swaziland', 'Zimbabwe'].


In [235]:
fancyCursor.execute('''
    SELECT a.country
    FROM 
    (SELECT lifeExp as "lifeExpA", country
    FROM data 
    WHERE year = "1952") a
    INNER JOIN
    (SELECT lifeEXP as "lifeExpB", country
    FROM data
    WHERE year = "2007") b
    ON a.country = b.country
    WHERE lifeExpA > lifeExpB
''')

records = [(row["country"]) for row in fancyCursor.fetchall()]

print(f"The countries whose life expectancy has decreased since 1952 are: {records}.")

The countries whose life expectancy has decreased since 1952 are: ['Swaziland', 'Zimbabwe'].


## Where are they now?

As of the new millenium (or, 2002, the closest year in the dataset), how many countries did each continent comprise?

In [90]:
populations = data[data["year"] == 2002][["country", "continent"]].groupby('continent')
records = [(cont, len(count)) for (cont, count) in populations.groups.items()]

for (continent, count) in records:
    print(f"{continent} has {count} countries.")

Africa has 52 countries.
Americas has 25 countries.
Asia has 33 countries.
Europe has 30 countries.
Oceania has 2 countries.


In [234]:
fancyCursor.execute('''
    SELECT continent, count(country) as "count"
    FROM data
    WHERE year = 2002
    GROUP BY continent
''')


records = [(row["continent"], row["count"]) for row in fancyCursor.fetchall()]

for (continent, count) in records:
    print(f"{continent} has {count} countries.")

Africa has 52 countries.
Americas has 25 countries.
Asia has 33 countries.
Europe has 30 countries.
Oceania has 2 countries.


## Best GDP year of the lowest GDP Country

Find the country with the lowest GDP in the dataset, and then report what their highest GDP has ever been, and what year that was.

In [141]:
lowestgdp = data.sort_values('gdpPercap', ascending = True).head(1)
nat = lowestgdp['country'].to_string(index=False).strip()
gdplist = data[data['country'] == nat]
srt = gdplist.sort_values('gdpPercap', ascending=False).head(1)[['country','gdpPercap','year']]

record = srt.to_dict('r')[0]

print(f"{record['country']}'s highest GDP Per Capita was {record['gdpPercap']:,.2f} in {record['year']}.")

Congo Dem. Rep.'s highest GDP Per Capita was 905.86 in 1957.


In [233]:
fancyCursor.execute('''
    SELECT country, gdpPercap, year
    FROM data
    WHERE country = 
        (SELECT country
        FROM data
        ORDER BY gdpPercap ASC
        LIMIT 1)
    ORDER BY gdpPercap DESC
    LIMIT 1
''')

record = {}

for row in fancyCursor.fetchall():
    record['country'] = row["country"]
    record['gdpPercap'] = row["gdpPercap"]
    record['year'] = row['year']

print(f"{record['country']}'s highest GDP Per Capita was {record['gdpPercap']:,.2f} in {record['year']}.")

Congo Dem. Rep.'s highest GDP Per Capita was 905.86 in 1957.


## Average Live Expectancy

What is the average life expectancy per continent across all years in the dataset?  Order the results by average life expectancy, descending.

In [217]:
srt = data[['continent','lifeExp']].groupby('continent').mean().sort_values('lifeExp',ascending=False)
records = list(zip(srt.index,srt['lifeExp']))

for (continent, avgLifeExp) in records:
    print(f"{continent} has an average life expectancy of {avgLifeExp:,.2f}.")

Oceania has an average life expectancy of 74.33.
Europe has an average life expectancy of 71.90.
Americas has an average life expectancy of 64.66.
Asia has an average life expectancy of 60.06.
Africa has an average life expectancy of 48.87.


In [232]:
fancyCursor.execute('''
    SELECT continent, avg(lifeExp) as "avgLifeExp"
    FROM data
    GROUP BY continent
    ORDER BY avgLifeExp DESC
''')

records = [(row["continent"], row["avgLifeExp"]) for row in fancyCursor.fetchall()]

for (continent, avgLifeExp) in records:
    print(f"{continent} has an average life expectancy of {avgLifeExp:,.2f}.")

Oceania has an average life expectancy of 74.33.
Europe has an average life expectancy of 71.90.
Americas has an average life expectancy of 64.66.
Asia has an average life expectancy of 60.06.
Africa has an average life expectancy of 48.87.


## Staying Alive

Which Country has had the largest percentage increase of life expectancy (`lifeExp`) from 1952 to 2007?

In [214]:
records1952 = data[data["year"] == 1952][["country","lifeExp"]].reset_index()
records2007 = data[data["year"] == 2007][["country", "lifeExp"]].reset_index()
percentincr = (records2007['lifeExp'] - records1952['lifeExp'])/records1952['lifeExp'] * 100
ind = percentincr.sort_values(ascending=False).index.values[0]

country = records1952.loc[ind]['country']
percent = percentincr[ind]

print(f"{country} had the largest percent increase in life expectancy, {percent:.1f}%")

Oman had the largest percent increase in life expectancy, 101.3%


In [231]:
fancyCursor.execute('''
    SELECT d.country, ((b.lifeExp - a.lifeExp)/(a.lifeExp)) * 100 AS "Percent"
    FROM data d
    INNER JOIN (
        SELECT country, lifeExp
        FROM data
        WHERE year = 1952) a
        ON d.country = a.country
    INNER JOIN (
        SELECT country, lifeExp
        FROM data
        WHERE year = 2007) b
        ON d.country = b.country
    GROUP BY d.country
    ORDER BY Percent DESC
    LIMIT 1
''')

country = ''
percent = 0.

result = {}

for row in fancyCursor.fetchall():
    country = row["country"]
    percent = row["Percent"]
    result['country'] = row["country"]
    result['percent'] = row["Percent"]

print(f"{result['country']} had the largest percent increase in life expectancy, {result['percent']:.1f}%")

Oman had the largest percent increase in life expectancy, 101.3%
